This notebook will contain transformations that are often used when doing ETL with data

#### Creating a unique key

When creating DIMENSION tables it's better to use a unique key and use that in the FACT table(s) later on.

In [ ]:
# disadvantage: a very long hashkey will be the unique key
hashKeys = [col("CustomerNo")]
df_CustomersT1 = df_CustomersT1.withColumn("uniqueHash", md5(concat_ws(",", *hashKeys)))

In [ ]:
# just a hash from the customer number, identified negative values here as well
df_CustomersT1 = df_CustomersT1.withColumn("CustomerKey", hash("CustomerNo"))

In [ ]:
from uuid import uuid5, NAMESPACE_OID
@udf
def create_deterministic_uuid(column):
    """
    Takes a column from a Pyspark Dataframe and returns the values as uuid5 hash.
    If None or 0, the value assigned is 0.
    """
    valuesMappedToZero = [None,"0",0,"_NA"]
    if column in valuesMappedToZero:
        return "0"
    else:
        return str(uuid5(NAMESPACE_OID, str(column)))

df_with_key = df.withColumn("ProductKey", create_deterministic_uuid(df["UniqueString"]))
